In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('../../dataset/meal_service/train.csv')

In [3]:
df_train.head()
df_train.describe()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,2807.815768,157.913693,241.142739,274.117012,43.506224,890.334440,461.772614
std,171.264404,144.190572,43.532298,246.239651,109.937400,209.505057,139.179202
min,2601.000000,23.000000,41.000000,0.000000,0.000000,296.000000,0.000000
25%,2645.000000,71.000000,217.000000,4.000000,0.000000,758.000000,406.000000
50%,2760.000000,105.000000,245.000000,299.000000,0.000000,879.000000,483.000000
75%,2962.000000,185.000000,272.000000,452.000000,0.000000,1032.000000,545.000000
max,3305.000000,1224.000000,378.000000,1044.000000,533.000000,1459.000000,905.000000


In [4]:
df_lunch_dinner = df_train[['일자', '중식메뉴', '중식계', '석식메뉴', '석식계']]
df_lunch_dinner.head()

,일자,중식메뉴,중식계,석식메뉴,석식계
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...",1039.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",331.0
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...",867.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",560.0
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...",1017.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",573.0
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...",978.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",525.0
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...",925.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",330.0


In [5]:
def seperate_and_processing_menu_str(menu_row, num_people, dish_cnt_dic, people_cnt_dic, menu_list):
    # first, split by space
    splits = menu_row.split(' ')
    menu = []
    for dish in splits:
        if len(dish) > 1:
            # find (New)
            if '(New)' in dish:
                menu.append(dish.split('(New)')[0] + dish.split('(New)')[1])
            elif '(' not in dish and ')' not in dish:
                menu.append(dish)
    if len(menu) > 3:
        menu_list.append(menu)
        for dish_str in menu:
            # for counting menu
            if dish_str in dish_cnt_dic:
                dish_cnt_dic[dish_str] = dish_cnt_dic[dish_str] + 1
            else:
                dish_cnt_dic[dish_str] = 1
                
            # for counting people
            if dish_str in people_cnt_dic:
                people_cnt_dic[dish_str] = people_cnt_dic[dish_str] + num_people
            else:
                people_cnt_dic[dish_str] = num_people
            
    else:
        # no menu today
        menu_list.append(['-'])

In [6]:
lunch_menu = []
dish_cnt_by_lunch_dish = {} # 'dish': lunch_menu_cnt cnt appeared in menu
people_cnt_by_lunch_dish = {} # 'dish': total_people_cnt who eat 'dish' at lunch
avg_people_cnt_by_lunch_dish = {} # 'dish': avg_people_cnt who eat 'dish' at lunch

dinner_menu = []
dish_cnt_by_dinner_dish = {} # 'dish': dinner_menu_cnt cnt appeared in menu
people_cnt_by_dinner_dish = {} # 'dish': total_people_cnt who eat 'dish' at dinne
avg_people_cnt_by_dinner_dish = {} # 'dish': avg_people_cnt who eat 'dish' at dinner

for idx in range(0, len(df_lunch_dinner)):
    org_row = df_lunch_dinner.iloc[idx]
    
    # lunch 
    row_lunch = org_row['중식메뉴']
    row_lunch_people_cnt = int(org_row['중식계'])
    seperate_and_processing_menu_str(row_lunch, row_lunch_people_cnt, dish_cnt_by_lunch_dish,
                                     people_cnt_by_lunch_dish, lunch_menu)
    
    # dinner
    row_dinner = org_row['석식메뉴']
    row_dinner_people_cnt = int(org_row['석식계'])
    seperate_and_processing_menu_str(row_dinner, row_dinner_people_cnt, dish_cnt_by_dinner_dish,
                                     people_cnt_by_dinner_dish, dinner_menu)



In [7]:
print(df_lunch_dinner['중식계'].mean())
print(df_lunch_dinner['석식계'].mean())

890.3344398340249
461.77261410788384


In [11]:
for key in dish_cnt_by_lunch_dish.keys():
    dish_cnt = dish_cnt_by_lunch_dish[key]
    people_cnt = people_cnt_by_lunch_dish[key]
    avg_people_cnt_by_lunch_dish[key] = people_cnt/dish_cnt

lunch_keys = list(avg_people_cnt_by_lunch_dish.keys())
lunch_sort_by_avg_people = sorted(lunch_keys, key = lambda x: dish_cnt_by_lunch_dish[x])

for menu in lunch_sort_by_avg_people:
    if (avg_people_cnt_by_lunch_dish[menu]) >= 1100 and dish_cnt_by_lunch_dish[menu] >= 3:
#         print('%-20s avg:%-10s cnt:%-5s' % (menu,
#                                             str(round(avg_people_cnt_by_lunch_dish[menu], 2)), 
#                                             str(dish_cnt_by_lunch_dish[menu])))
        print('\'%s\',' % (menu))

'새송이버섯조림',
'풋마늘초무침',
'맛살겨자채',
'한방소갈비찜',
'청경채사과생채',
'포기김치/요구르트',
'포기김치/바나나',
'쫄면채소무침',
'모둠쌈',
'돈육김치볶음',
'돈육간장불고기',
'모듬쌈*쌈장',
'소불고기',


In [9]:
for key in dish_cnt_by_dinner_dish.keys():
    dish_cnt = dish_cnt_by_dinner_dish[key]
    people_cnt = people_cnt_by_dinner_dish[key]
    avg_people_cnt_by_dinner_dish[key] = people_cnt/dish_cnt

dinner_keys = list(avg_people_cnt_by_dinner_dish.keys())
dinner_sort_by_avg_people = sorted(dinner_keys, key = lambda x: avg_people_cnt_by_dinner_dish[x])

for menu in dinner_sort_by_avg_people:
    if (avg_people_cnt_by_dinner_dish[menu]) >= 560 and dish_cnt_by_dinner_dish[menu] >= 3:
        print('%-20s avg:%-10s cnt:%-5s' % (menu,
                                            str(round(avg_people_cnt_by_dinner_dish[menu], 2)), 
                                            str(dish_cnt_by_dinner_dish[menu])))
        #print('\'%s\',' % (menu))

'수박',
'락교',
'우동',
'로제스파게티',
'오므라이스',
'후난식볶음밥',
'통도라지구이',
'미니햄버거',
'김치볶음밥',
'가쯔오장국',
'콘샐러드',
'무채국',
'해파리냉채',
'콥샐러드',
'취나물',
'비트무생채',


In [10]:
'''
Todo
식사인원이 특정값 이상일 때의 메뉴별 통계
cnt가 특정값 이상일 때의 통계

Conclusion
생각보다 

'''

'\nTodo\n식사인원이 특정값 이상일 때의 메뉴별 통계\ncnt가 특정값 이상일 때의 통계\n\nConclusion\n생각보다 \n\n'